In [33]:
import tensorflow as tf

In [46]:
import numpy as np
from keras.models import Sequential
from tensorflow.keras import metrics
from keras.layers import Dense, activation
from keras.applications import VGG16
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from PIL import Image

In [32]:
vgg_model = VGG16()
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [34]:
last_layer = str(vgg_model.layers[-1])

seed = 42
np.random.seed(seed)

classifier=Sequential()


for layer in vgg_model.layers:
    if str(layer) != last_layer:
        classifier.add(layer)

In [35]:
for layer in classifier.layers:
    layer.trainable=False

In [36]:
classifier.add(Dense(1,activation="sigmoid"))
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [102]:
classifier.compile(loss="binary_crossentropy",metrics="f1score",optimizer="adam")

In [106]:
import tensorflow as tf
from keras import backend as K

def f1score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1_score = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1_score)

classifier.compile(loss='binary_crossentropy', metrics=[f1score], optimizer='adam')


In [107]:
train_deta = ImageDataGenerator(rescale=1./225,
                               shear_range=.2,
                               horizontal_flip=True,
                               zoom_range=.2)

test_data = ImageDataGenerator(rescale=1./225)

In [108]:
training_data = train_deta.flow_from_directory("C:/Users/97155/Downloads/images/images/training",
                                               target_size = (224,224),
                                               batch_size=32,
                                               class_mode = "binary"
                                                )

testing_data = test_data.flow_from_directory("C:/Users/97155/Downloads/images/images/testing",
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode="binary"
                                            )

Found 2392 images belonging to 2 classes.
Found 597 images belonging to 2 classes.


In [109]:
classifier.fit_generator(training_data,epochs=10,validation_data=testing_data)

C:\Users\97155\AppData\Local\Temp\ipykernel_9740\4125818525.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_data,epochs=10,validation_data=testing_data)


Epoch 1/10
75/75 [==============================] - 568s 8s/step - loss: 0.6917 - f1score: 0.5424 - val_loss: 0.6325 - val_f1score: 0.5451
Epoch 2/10
75/75 [==============================] - 519s 7s/step - loss: 0.6102 - f1score: 0.6355 - val_loss: 0.5723 - val_f1score: 0.7349
Epoch 3/10
75/75 [==============================] - 524s 7s/step - loss: 0.5695 - f1score: 0.6914 - val_loss: 0.5427 - val_f1score: 0.7508
Epoch 4/10
75/75 [==============================] - 507s 7s/step - loss: 0.5649 - f1score: 0.7075 - val_loss: 0.5093 - val_f1score: 0.7634
Epoch 5/10
75/75 [==============================] - 514s 7s/step - loss: 0.5080 - f1score: 0.7756 - val_loss: 0.4944 - val_f1score: 0.7729
Epoch 6/10
75/75 [==============================] - 506s 7s/step - loss: 0.4973 - f1score: 0.7821 - val_loss: 0.4882 - val_f1score: 0.7645
Epoch 7/10
75/75 [==============================] - 506s 7s/step - loss: 0.4889 - f1score: 0.7784 - val_loss: 0.4553 - val_f1score: 0.8106
Epoch 8/10
75/75 [=========

In [186]:
new_image = Image.open("C:/Users/97155/Downloads/WhatsApp Image 2023-06-27 at 11.45.44 AM.jpeg")

In [187]:
new_image = new_image.resize((224, 224))
new_image = np.array(new_image)
new_image = np.expand_dims(new_image, axis=0)

In [188]:
y_pred=classifier.predict(new_image)

if y_pred >.5:
    print("NOT FLIP")
else:
    print("FLIP")


1/1 [==============================] - 0s 198ms/step
FLIP


In [189]:
y_pred

array([[0.02131842]], dtype=float32)

In [167]:
new_image1=Image.open("C:/Users/97155/Downloads/WhatsApp Image 2023-06-27 at 11.40.10 AM.jpeg")
new_image2=Image.open("C:/Users/97155/Downloads/WhatsApp Image 2023-06-27 at 11.40.10 AM (1).jpeg")
new_image3=Image.open("C:/Users/97155/Downloads/WhatsApp Image 2023-06-27 at 11.40.09 AM.jpeg")
new_image4=Image.open("C:/Users/97155/Downloads/WhatsApp Image 2023-06-27 at 11.45.44 AM.jpeg")
new_image5=Image.open("C:/Users/97155/Downloads/WhatsApp Image 2023-06-27 at 11.47.36 AM.jpeg")

In [159]:
images=[new_image1,new_image2,new_image3,new_image4,new_image5]

In [160]:
pred = []
for i in images:
    i = i.resize((224, 224))
    i = np.array(i)
    i = np.expand_dims(i, axis=0)
    y_pred = classifier.predict(i)

    if y_pred[0] > 0.5:  
        a = "NOT FLIP"
    else:
        a = "FLIP"

    pred.append(a)


    

1/1 [==============================] - 0s 213ms/step


In [161]:
pred

['NOT FLIP', 'NOT FLIP', 'NOT FLIP', 'FLIP', 'FLIP']